In [ ]:
!pip install -q tensorflow-model-optimization

In [1]:
import tempfile
import os

import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
# Load data
student_df = pd.read_csv('../../datasets/student_mat_processed01.csv')

train_dataset = student_df.sample(frac=0.8, random_state=0)
test_dataset = student_df.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_G1_targets = train_features.pop('G1')
train_G2_targets = train_features.pop('G2')
train_G3_targets = train_features.pop('G3')

test_G1_targets = test_features.pop('G1')
test_G2_targets = test_features.pop('G2')
test_G3_targets = test_features.pop('G3')

In [3]:
# Normalizer Layer
normalizer = preprocessing.Normalization()
normalizer.adapt(np.array(train_features))

In [4]:
# Define the model architecture.
def build_and_compile_model():
    model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(45)),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(1)
    ])

    # Train the model
    model.compile(loss='mean_absolute_error', metrics=[tf.keras.metrics.MeanAbsoluteError()],
                  optimizer=tf.keras.optimizers.Adam(0.001))
    
    return model

In [5]:
fc_model = build_and_compile_model()
fc_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                1472      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 2,145
Trainable params: 2,145
Non-trainable params: 0
_________________________________________________________________


In [58]:
history = fc_model.fit(
    train_features, train_G3_targets,
    validation_split=0.2,
    verbose=1, epochs=100)

Epoch 1/100
8/8 [==============================] - 0s 21ms/step - loss: 0.7990 - mean_absolute_error: 0.7990 - val_loss: 3.7247 - val_mean_absolute_error: 3.7247
Epoch 2/100
8/8 [==============================] - 0s 6ms/step - loss: 0.8140 - mean_absolute_error: 0.8140 - val_loss: 3.8510 - val_mean_absolute_error: 3.8510
Epoch 3/100
8/8 [==============================] - 0s 7ms/step - loss: 0.7610 - mean_absolute_error: 0.7610 - val_loss: 3.7407 - val_mean_absolute_error: 3.7407
Epoch 4/100
8/8 [==============================] - 0s 8ms/step - loss: 0.6594 - mean_absolute_error: 0.6594 - val_loss: 3.6565 - val_mean_absolute_error: 3.6565
Epoch 5/100
8/8 [==============================] - 0s 7ms/step - loss: 0.6556 - mean_absolute_error: 0.6556 - val_loss: 3.6599 - val_mean_absolute_error: 3.6599
Epoch 6/100
8/8 [==============================] - 0s 6ms/step - loss: 0.6063 - mean_absolute_error: 0.6063 - val_loss: 3.6606 - val_mean_absolute_error: 3.6606
Epoch 7/100
8/8 [================

8/8 [==============================] - 0s 6ms/step - loss: 0.3728 - mean_absolute_error: 0.3728 - val_loss: 3.8071 - val_mean_absolute_error: 3.8071
Epoch 52/100
8/8 [==============================] - 0s 7ms/step - loss: 0.3628 - mean_absolute_error: 0.3628 - val_loss: 3.8358 - val_mean_absolute_error: 3.8358
Epoch 53/100
8/8 [==============================] - 0s 6ms/step - loss: 0.3611 - mean_absolute_error: 0.3611 - val_loss: 3.8069 - val_mean_absolute_error: 3.8069
Epoch 54/100
8/8 [==============================] - 0s 6ms/step - loss: 0.3835 - mean_absolute_error: 0.3835 - val_loss: 3.8771 - val_mean_absolute_error: 3.8771
Epoch 55/100
8/8 [==============================] - 0s 5ms/step - loss: 0.3546 - mean_absolute_error: 0.3546 - val_loss: 3.8102 - val_mean_absolute_error: 3.8102
Epoch 56/100
8/8 [==============================] - 0s 6ms/step - loss: 0.3569 - mean_absolute_error: 0.3569 - val_loss: 3.8183 - val_mean_absolute_error: 3.8183
Epoch 57/100
8/8 [=======================

In [59]:
test_results = {}
_, test_results['fc_model'] = fc_model.evaluate(test_features, test_G3_targets, verbose=0)

print('Baseline mean absolute error:', test_results['fc_model'])

#_, keras_file = tempfile.mkstemp('.h5')
#tf.keras.models.save_model(fc_model, keras_file, include_optimizer=False)
#print('Saved baseline model to:', keras_file)

Baseline mean absolute error: 3.8007121086120605


In [60]:
import tensorflow_model_optimization as tfmot
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

In [61]:
# Compute end step to finish pruning after 2 epochs.
batch_size = 32
epochs = 8
validation_split = 0.1 # 10% of training set will be used for validation set.

num_features = train_features.shape[0] * (1 - validation_split)
end_step = np.ceil(num_features / batch_size).astype(np.int32) * epochs

In [62]:
# Define model for pruning.
pruning_params = {
    'pruning_schedule':
    tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                         final_sparsity=0.80,
                                         begin_step=0,
                                         end_step=end_step)
}

model_for_pruning = prune_low_magnitude(fc_model, **pruning_params)

In [63]:
# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='mean_absolute_error',
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_dense (P (None, 32)                2914      
_________________________________________________________________
prune_low_magnitude_dense_1  (None, 16)                1042      
_________________________________________________________________
prune_low_magnitude_dense_2  (None, 8)                 266       
_________________________________________________________________
prune_low_magnitude_dense_3  (None, 1)                 19        
Total params: 4,241
Trainable params: 2,145
Non-trainable params: 2,096
_________________________________________________________________


In [64]:
# Training and evaluate the model against baseline
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_features, train_G3_targets,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/8
2/9 [=====>........................] - ETA: 8s - loss: 1.0782 - mean_absolute_error: 1.0782WARNING:tensorflow:Callbacks method `on_train_batch_begin` is slow compared to the batch time (batch time: 0.0027s vs `on_train_batch_begin` time: 0.0081s). Check your callbacks.


9/9 [==============================] - 3s 315ms/step - loss: 1.0374 - mean_absolute_error: 1.0374 - val_loss: 3.6414 - val_mean_absolute_error: 3.6414
Epoch 2/8
9/9 [==============================] - 0s 7ms/step - loss: 0.9807 - mean_absolute_error: 0.9807 - val_loss: 3.6298 - val_mean_absolute_error: 3.6298
Epoch 3/8
9/9 [==============================] - 0s 7ms/step - loss: 0.9494 - mean_absolute_error: 0.9494 - val_loss: 3.6124 - val_mean_absolute_error: 3.6124
Epoch 4/8
9/9 [==============================] - 0s 7ms/step - loss: 0.9324 - mean_absolute_error: 0.9324 - val_loss: 3.5982 - val_mean_absolute_error: 3.5982
Epoch 5/8
9/9 [==============================] - 0s 7ms/step - loss: 0.9393 - mean_absolute_error: 0.9393 - val_loss: 3.6269 - val_mean_absolute_error: 3.6269
Epoch 6/8
9/9 [==============================] - 0s 7ms/step - loss: 0.9207 - mean_absolute_error: 0.9207 - val_loss: 3.6194 - val_mean_absolute_error: 3.6194
Epoch 7/8
9/9 [==============================] - 0s 7m

In [65]:
_, test_results['pruned_model'] = model_for_pruning.evaluate(
   test_features, test_G3_targets, verbose=1)

print('Baseline mean absolute error:', test_results['fc_model']) 
print('Pruned mean absolute error:', test_results['pruned_model'])

3/3 [==============================] - 0s 2ms/step - loss: 3.7642 - mean_absolute_error: 3.7642
Baseline mean absolute error: 3.8007121086120605
Pruned mean absolute error: 3.7642366886138916


In [14]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export,
                           pruned_keras_file,
                           include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

Saved pruned Keras model to: C:\Users\z0042fkb\AppData\Local\Temp\tmpoxz5ghbm.h5


In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\z0042fkb\AppData\Local\Temp\tmpk_2tfhrs\assets
Saved pruned TFLite model to: C:\Users\z0042fkb\AppData\Local\Temp\tmpz03bcosp.tflite


In [16]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [17]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 9886.00 bytes
Size of gzipped pruned Keras model: 6823.00 bytes
Size of gzipped pruned TFlite model: 5941.00 bytes


In [39]:
#!pip install -U tensorboard
#%load_ext tensorboard
#%tensorboard --logdir={logdir}
#print(logdir)
#%tensorboard dev upload --logdir 'C:\\Users\\z0042fkb\\AppData\\Local\\Temp\\tmppg6etlnp'
#%tensorboard dev auth revoke

ERROR: Failed to launch TensorBoard (exited with 0).
Contents of stderr:
2021-04-27 17:50:53.881608: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'cudart64_101.dll'; dlerror: cudart64_101.dll not found
2021-04-27 17:50:53.890311: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Logged out of uploader.